# Reconstruct the SBML from a set of reactions

This is to test the FBA toolchain

In [1]:
import sys
import os
import copy
import PyFBA
import re
import pickle

import inspect
inspect.getfile(PyFBA)

'/home/redwards/GitHubsLinux/PyFBA/venv/lib/python3.9/site-packages/PyFBA-2.2-py3.9.egg/PyFBA/__init__.py'

# Read the Reactions, Enzymes, and Compounds Sets

This parses the ModelSeed data

In [2]:
modeldata = PyFBA.parse.model_seed.parse_model_seed_data('gramnegative', verbose=True)
print(f"There are {len(modeldata.compounds):,} compounds, {len(modeldata.reactions):,} reactions, and {len(modeldata.enzymes):,} enzymes in total")

We are logging to /home/redwards/GitHubsLinux/PyFBA/iPythonNotebooks/logs/PyFBA.2021-06-14T12:06:44.123160.log
Reading compounds from PyFBA.Biochemistry.ModelSEEDDatabase.Biochemistry.compounds.json
Reading reactions from PyFBA.Biochemistry.ModelSEEDDatabase.Biochemistry.reactions.json
Creating enzymes with complexes and reactions


There are 33,845 compounds, 43,774 reactions, and 9,423 enzymes in total


Modelseed.compounds is entirely `PyFBA.metabolism.Compound` and does not have locations.

# Create a biomass equation

In [3]:
biomass_equation = PyFBA.metabolism.biomass_equation('gramnegative', modeldata.compounds)

# Read the media and normalize any of the compound names

In [4]:
# Read the media file
media = PyFBA.parse.pyfba_media("ArgonneLB")
# Correct the names
media = PyFBA.parse.correct_media_names(media, modeldata.compounds)
print(f"The media has {len(media)} compounds")

The media has 65 compounds


`media` is a set of `PyFBA.metabolism.CompoundWithLocation`

# Read the reactions

In [17]:
reactions_to_run = set()
with open('rgood.txt', 'r') as f:
    for l in f:
        if l.startswith('rxn'):
            reactions_to_run.add(l.strip())
print(f"There are {len(reactions_to_run)} reactions to run")

There are 1399 reactions to run


Note that we do not need to add the biomass reaction to reactions_to_run as that is parsed separately when we create the stiochiometric matrix.

# Run the FBA

In [19]:
uptake_secretion_reactions = None
status, value, growth = PyFBA.fba.run_fba(modeldata, reactions_to_run, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=False)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


create_stoichiometric_matrix found 190 uptake and secretion reactions


The FBA completed with a flux value of 933.3130068801794 --> growth: True


In [7]:
cpd = {}
ecpd = {}
for r in reactions_to_run:
    for c in modeldata.reactions[r].all_compounds():
        cpd[c] = cpd.get(c, 0) + 1
        if c.location == 'e':
            ecpd[c] = ecpd.get(c, 0) + 1
print(f"There are {len(cpd)} unique compounds")
print(f"There are {len(ecpd)} external compounds")

There are 1302 unique compounds
There are 172 external compounds


In [8]:
for m in media:
    cpd[m] = cpd.get(m, 0) + 1
    if c.location == 'e':
        ecpd[c] = ecpd.get(c, 0) + 1
print(f"There are {len(cpd)} unique compounds")
print(f"There are {len(ecpd)} external compounds")

There are 1319 unique compounds
There are 172 external compounds


In [9]:
for e in ecpd:
    newid = f"Exhaust_{e.id}"
    newr = PyFBA.metabolism.Reaction(newid, newid)
    eb = PyFBA.metabolism.CompoundWithLocation.from_compound(e, 'b')
    newr.add_left_compounds({e})
    newr.set_left_compound_abundance(e, 1)
    newr.add_right_compounds({eb})
    newr.set_right_compound_abundance(eb, 1)
    newr.equation = f'(1) {e} = (1) {eb}'
    newr.direction = '='
    modeldata.reactions[newid] = newr
    reactions_to_run.add(newid)

In [10]:
uptake_secretion_reactions = None
status, value, growth = PyFBA.fba.run_fba(modeldata, reactions_to_run, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

csm: after adding 65 media, cp: 65, sm has 65
csm: after adding 1571 r2r, cp: 1491, sm has 1491
csm: after adding biomass, cp: 1492, sm has 1492
create_stoichiometric_matrix found 190 uptake and secretion reactions
csm: before adding 190 upsr, cp x rxn: 1492 x 1571
csm: after adding 190 upsr, cp x rxn: 1492 x 1761
csm: In the model there are : 1492 compounds and 1762 reactions
We are loading 1492 rows and 1762 columns


The FBA completed with a flux value of 933.3130068801801 --> growth: True


csm: sm will be compounds 1492 x reactions 1762
Length of the media: 65
Number of reactions to run: 1571
Number of compounds in SM: 1492
Number of reactions in SM: 1762
Number of uptake/secretion reactions 190
Revised number of total reactions: 43946
Revised number of total compounds: 33845
SMat dimensions: 1492 x 1762


## Gapfilling the reactions

In [11]:
added_reactions = []
reactionsource = {r: "original" for r in reactions_to_run}

In [12]:
def gap_fill_media():
    sys.stderr.write("Gap filling from MEDIA\n")
    media_reactions = PyFBA.gapfill.suggest_from_media(modeldata, reactions_to_run, media, verbose=True)
    added_reactions.append(("media", media_reactions))
    reactions_to_run.update(media_reactions)
    status, value, growth = PyFBA.fba.run_fba(modeldata,
                                          reactions_to_run, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
    sys.stderr.write(f"After adding {len(media_reactions)} MEDIA reactions we get {value} (growth is {growth})\n")
    for r in media_reactions:
        if r not in reactionsource:
            reactionsource[r] = 'media_reactions'

    if growth:
        additions = resolve_additional_reactions(original_reactions, added_reactions, compounds, reactions,
                                                 media, biomass_eqtn)
        # print('reactions' + " : " + str(original_reactions.union(additions)))
        for r in original_reactions.union(additions):
            if r not in reactionsource:
                reactionsource[r] = "UNKNOWN??"
            print("{}\t{}".format(r, reactionsource[r]))

In [13]:
def essential_reactions():
    sys.stderr.write("Gap filling from ESSENTIAL PROTEINS\n")
    essential_reactions = PyFBA.gapfill.suggest_essential_reactions()
    # find only the new reactions
    essential_reactions.difference_update(reactions_to_run)
    added_reactions.append(("essential", essential_reactions))
    reactions_to_run.update(essential_reactions)
    status, value, growth = PyFBA.fba.run_fba(modeldata,
                                      reactions_to_run, media, biomass_equation,
                                      uptake_secretion_reactions, verbose=True)
    sys.stderr.write("After adding {} ESSENTIAL reactions we get {} (growth is {})\n\n".format(len(essential_reactions),
                                                                                               value, growth))

    for r in essential_reactions:
        if r not in reactionsource:
            reactionsource[r] = 'essential_reactions'

    # if this grows then we want to find the minimal set of reactions
    # that we need to add for growth and call it good.
    if growth:
        additions = resolve_additional_reactions(original_reactions, added_reactions, modeldata.compounds, modeldata.reactions,
                                                 media, biomass_equation)
        for r in original_reactions.union(additions):
            if r not in reactionsource:
                reactionsource[r] = "UNKNOWN??"
            print("{}\t{}".format(r, reactionsource[r]))

In [14]:
def close_genomes(close_genomes=None, similar_genera=None):
    sys.stderr.write("Gap filling from CLOSE GENOMES\n")
    close_reactions = set()
    if close_genomes:
        # add reactions from roles in close genomes
        close_reactions = PyFBA.gapfill.suggest_from_roles(close_genomes, modeldata.reactions, threshold=0, verbose=True)
        # find the new reactions
        close_reactions.difference_update(reactions_to_run)
        added_reactions.append(("close genomes ", close_reactions))
        reactions_to_run.update(close_reactions)
        status, value, growth = PyFBA.fba.run_fba(modeldata,
                                  reactions_to_run, media, biomass_equation,
                                  uptake_secretion_reactions, verbose=True)
        
        sys.stderr.write(f"After adding {len(close_reactions)} reactions in {close_genomes} we get {value} (growth is {growth})\n")
        
        for r in close_reactions:
            if r not in reactionsource:
                reactionsource[r] = 'close_reactions'

        # if this grows then we want to find the minimal set of reactions
        # that we need to add for growth and call it good.
        if growth:
            additions = resolve_additional_reactions(original_reactions, added_reactions, modeldata.compounds, modeldata.reactions,
                                                     media, biomass_equation)
            # print("Additional reactions required: " + str(additions) + "\n")
            # print("'reactions': {}".format(original_reactions.union(additions)))
            for r in original_reactions.union(additions):
                if r not in reactionsource:
                    reactionsource[r] = "UNKNOWN??"
                print("{}\t{}".format(r, reactionsource[r]))

    genus_reactions = set()
    if similar_genera:
        # add reactions from roles in similar genera
        genus_reactions = PyFBA.gapfill.suggest_from_roles(similar_genera, reactions, threshold=0, verbose=True)
        # find the new reactions
        genus_reactions.difference_update(reactions_to_run)
        added_reactions.append(("other genera", genus_reactions))
        reactions_to_run.update(genus_reactions)
        status, value, growth = PyFBA.fba.run_fba(modeldata,
                          reactions_to_run, media, biomass_equation,
                          uptake_secretion_reactions, verbose=True)        
        sys.stderr.write(f"After adding {len(genus_reactions)} reactions in {similar_genera} we get {value} (growth is {growth})\n")

        for r in genus_reactions:
            if r not in reactionsource:
                reactionsource[r] = 'genus_reactions'

        # if this grows then we want to find the minimal set of reactions
        # that we need to add for growth and call it good.
        if growth:
            additions = resolve_additional_reactions(original_reactions, added_reactions, modeldata.compounds, modeldata.reactions,
                                                     media, biomass_equation)
            for r in original_reactions.union(additions):
                if r not in reactionsource:
                    reactionsource[r] = "UNKNOWN??"
                print("{}\t{}".format(r, reactionsource[r]))


In [15]:
def gapfill_subsystems():
    sys.stderr.write("Gap filling from SUBSYSTEMS\n")
    subsystem_reactions = PyFBA.gapfill.suggest_reactions_from_subsystems(
        modeldata.reactions, reactions_to_run, threshold=0.5, verbose=True)
    added_reactions.append(("subsystems", subsystem_reactions))
    reactions_to_run.update(subsystem_reactions)
    status, value, growth = PyFBA.fba.run_fba(modeldata, reactions_to_run, media, biomass_equation)
    sys.stderr.write(f"After adding {len(subsystem_reactions)} SUBSYSTEM reactions we get {value} (growth is {growth})\n")
    for r in subsystem_reactions:
        if r not in reactionsource:
            reactionsource[r] = 'subsystem_reactions'

    if growth:
        additions = resolve_additional_reactions(original_reactions, added_reactions, modeldata.compounds, modeldata.reactions,
                                                 media, biomass_equation)
        # print('reactions' + " : " + str(original_reactions.union(additions)))
        for r in original_reactions.union(additions):
            if r not in reactionsource:
                reactionsource[r] = "UNKNOWN??"
            print("{}\t{}".format(r, reactionsource[r]))

In [16]:
essential_reactions()

Gap filling from ESSENTIAL PROTEINS
csm: after adding 65 media, cp: 65, sm has 65
csm: after adding 1574 r2r, cp: 1492, sm has 1492
csm: after adding biomass, cp: 1493, sm has 1493
create_stoichiometric_matrix found 190 uptake and secretion reactions
csm: before adding 190 upsr, cp x rxn: 1493 x 1574
csm: after adding 190 upsr, cp x rxn: 1493 x 1764
csm: In the model there are : 1493 compounds and 1765 reactions
We are loading 1493 rows and 1765 columns
csm: sm will be compounds 1493 x reactions 1765
Length of the media: 65
Number of reactions to run: 1574
Number of compounds in SM: 1493
Number of reactions in SM: 1765
Number of uptake/secretion reactions 190
Revised number of total reactions: 43946
Revised number of total compounds: 33845
SMat dimensions: 1493 x 1765
After adding 3 ESSENTIAL reactions we get 933.3130068801797 (growth is True)



NameError: name 'resolve_additional_reactions' is not defined

In [ ]:
gapfill_subsystems()

In [ ]:
gap_fill_media()

## Comparing to the SBML file that does grow!

We pickled that data, and can load it here. 

In [ ]:
sbml_filtered_compounds = pickle.load(open('compounds.pickle', 'rb'))
sbml_reactions = pickle.load(open('reactions.pickle', 'rb'))
sbml_reactions_to_run = pickle.load(open('reactions_to_run.pickle', 'rb'))
sbml_media = pickle.load(open('media.pickle', 'rb'))
sbml_biomass_equation = pickle.load(open('sbml_biomass.pickle', 'rb'))
sbml_uptake_secretion_reactions = pickle.load(open('uptake_secretion_reactions.pickle', 'rb'))

In [ ]:
for r in sbml_reactions:
    for c in sbml_reactions[r].all_compounds():
        if '_' in c.name:
            print(c.name)

In [ ]:
sbml_by_equation = set()
for r in sbml_reactions:
    sbml_by_equation.add(sbml_reactions[r].equation)
ms_by_equation = set()
for r in modeldata.reactions:
    ms_by_equation.add(modeldata.reactions[r].equation)


In [ ]:
print(f"There are {len(sbml_by_equation)} sbml reactions and {len(ms_by_equation)} model seed reactions")
print(f"{len(sbml_by_equation.intersection(ms_by_equation))} reactions are in both")

In [ ]:
print(f"There are {len(sbml_reactions_to_run)} sbml reactions to run and {len(reactions_to_run)} model seed reactions to run")
print(f"{len(sbml_reactions_to_run.intersection(reactions_to_run))} are the same")


In [ ]:
# choose an sbml reaction to run at random
from random import sample

In [ ]:
rr = sample(list(sbml_reactions_to_run), 1)[0]
if rr in reactions_to_run:
    print(rr)
    print(sbml_reactions[rr].equation)
    print(modeldata.reactions[rr].equation)

In [ ]:
def fixname(x):
    x = x.replace('-', '_')
    x = x.replace(',', '_')
    x = x.replace('sn_glycerol 3_phosphate', 'sn_glycerol_3_phosphate')
    return x

sbml_rewritten = {}
for rx in sbml_reactions:
    l = ""
    for c in sorted(sbml_reactions[rx].left_compounds, key=lambda c:c.name.lower()):
        if c.name == 'H' or c.name == 'H+':
            continue
        l += f" + ({sbml_reactions[rx].get_left_compound_abundance(c):.0f}) {fixname(c.name)}[{c.location}]"
    l = l.replace(' + ', '', 1)
    r = ""
    for c in sorted(sbml_reactions[rx].right_compounds, key=lambda c:c.name.lower()):
        if c.name == 'H' or c.name == 'H+':
            continue
        r += f" + ({sbml_reactions[rx].get_right_compound_abundance(c):.0f}) {fixname(c.name)}[{c.location}]"
    r = r.replace(' + ', '', 1)
    e = f"{l} {sbml_reactions[rx].direction} {r}"
    sbml_rewritten[rx] = e

ms_rewritten = {}
for rx in modeldata.reactions:
    l = ""
    for c in sorted(modeldata.reactions[rx].left_compounds, key=lambda c:c.name.lower()):
        if c.name == 'H' or c.name == 'H+':
            continue
        l += f" + ({modeldata.reactions[rx].get_left_compound_abundance(c):.0f}) {fixname(c.name)}[{c.location}]"
    l = l.replace(' + ', '', 1)
    r = ""
    for c in sorted(modeldata.reactions[rx].right_compounds, key=lambda c:c.name.lower()):
        if c.name == 'H' or c.name == 'H+':
            continue
        r += f" + ({modeldata.reactions[rx].get_right_compound_abundance(c):.0f}) {fixname(c.name)}[{c.location}]"
    r = r.replace(' + ', '', 1)
    e = f"{l} {modeldata.reactions[rx].direction} {r}"
    ms_rewritten[rx] = e

In [ ]:
for r in sbml_rewritten:
    if r in ms_rewritten:
        if sbml_rewritten[r] != ms_rewritten[r]:
            print(f"\n{r}\ns: {sbml_rewritten[r]}\nm: {ms_rewritten[r]}")

So if we use the `sbml_reactions` and the `sbml_biomass_equation`, we get growth. Without those reactions we get an error. With the normal biomass equation, we don't get any growth

### Note the normal biomass equation does not work!

We need to figure out what is missing!

In [ ]:
biomass_equation = PyFBA.metabolism.biomass_equation('gram_negative')
status, value, growth = PyFBA.fba.run_fba(modeldata,
                                          reactions_to_run, media, biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

### And the modeldata.reactions gives us an error (or no growth

In [ ]:
status, value, growth = PyFBA.fba.run_fba(modeldata,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
for r in sbml_reactions:
    if r not in modeldata.reactions:
        print(r)

In [ ]:
tmpr = copy.deepcopy(modeldata.reactions)
tmpr['biomass_equation'] = sbml_reactions['biomass_equation']

In [ ]:
status, value, growth = PyFBA.fba.run_fba(modeldata,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
sbml_reactions['EX_cpd01022'].__dict__

In [ ]:
external = set()
for r in reactions_to_run:
    for c in modeldata.reactions[r].all_compounds():
        if c.location == 'e':
            external.add(c)
print(f"There are {len(external)} external compounds")

In [ ]:
newr = {}
for e in external:
    n = PyFBA.metabolism.Reaction(f"EX_{e.id}", readable_name='external rctn {e.id}',
                                 equation = f"(1) {e} = (1) {e}", direction='=')
    n.add_left_compounds(c)
    n.add_right_compounds(c)
    n.set_left_compound_abundance(c, 1)
    n.set_right_compound_abundance(c, 1)
    n.lower_bound = -1000
    n.upper_bound = 1000
    newr[f"EX_{e.id}"] = n


In [ ]:
mr = copy.deepcopy(modeldata.reactions)
mr.update(newr)
status, value, growth = PyFBA.fba.run_fba(modeldata,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
mr['biomass_equation'] = sbml_reactions['biomass_equation']
status, value, growth = PyFBA.fba.run_fba(modeldata,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
for r in sbml_reactions:
    if r not in mr:
        print(r)

In [ ]:
sbml_reactions['EX_cpd08636'].__dict__

In [ ]:
sbml_partial = copy.deepcopy(sbml_reactions)
status, value, growth = PyFBA.fba.run_fba(modeldata,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
del sbml_partial['EX_cpd11416']

In [ ]:
for f in ['EX_cpd15302', 'EX_cpd08636', 'EX_cpd02701']:
    del sbml_partial[f]

In [ ]:
for r in sbml_partial:
    if r not in mr:
        print(r)

In [ ]:
sbml_partial = copy.deepcopy(sbml_reactions)
status, value, growth = PyFBA.fba.run_fba(modeldata,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
for r in sbml_reactions:
    if r in modeldata.reactions:
        left_diff = 0
        right_diff = 0
        for l in modeldata.reactions[r].left_compounds:
            try:
                a = sbml_reactions[r].get_left_compound_abundance(l)
            except KeyError:
                left_diff += 1
        for l in modeldata.reactions[r].right_compounds:
            try:
                a = sbml_reactions[r].get_right_compound_abundance(l)
            except KeyError:
                right_diff += 1
        if left_diff > 0 or right_diff > 0:
            print(f"{r}\t{left_diff}\t{right_diff}")

In [ ]:
for r in sbml_reactions:
    print(r)

In [ ]:
sbml_reactions['rxn01516'].__dict__

In [ ]:
modeldata.reactions['rxn01516'].__dict__

In [ ]:
for r in reactions_to_run:
    for c in modeldata.reactions[r].left_compounds:
        print(f"left {c.id} abundance {0 - modeldata.reactions[r].get_left_compound_abundance(c)}")


In [ ]:
modeldata.get_compound_by_id('cpd00037').name

In [ ]:
for c in modeldata.compounds:
    if c.id == 'cpd00060':
        if isinstance(c, PyFBA.metabolism.CompoundWithLocation):
            print(f"{c} ({c.id}) ({c.name}) ({c.location}): {c.__hash__()}")
            print(f"{hash((c.id, c.name, c.location))}")
        else:
            print(f"{c} ({c.id}) ({c.name}): {c.__hash__()}")